In [27]:
import numpy as np
import pandas as pd
import os
import shutil
import shlex
import sys
import subprocess as sp
import time
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import time
sys.path.append('./src/')
from utilities import *


def main():
    if(len(sys.argv) <= 1):
        print('Usage:\n\
               python ./src/run.py [data | data-test] process')
    for arg in sys.argv:
        if arg == 'data':
            data(**(json.load(open('config/data.json'))))
        elif arg == 'data-test':
            data_test(**(json.load(open('config/data-test.json'))))
        elif arg == 'process':
            process(**(json.load(open('config/process.json'))))
    
def data(fastq_files, bam_files, vcf_files, ref_file, datapath):
    # Create data directories
    if not os.path.exists(datapath):
        os.mkdir(datapath)
    if not os.path.exists(datapath + 'fastq_files/'):
        os.mkdir(datapath + 'fastq_files/')
    if not os.path.exists(datapath + 'bam_files/'):
        os.mkdir(datapath + 'bam_files/')
    if not os.path.exists(datapath + 'vcf_files/'):
        os.mkdir(datapath + 'vcf_files/')
    
    # Copy or download data
    procs = []
    print("Copying FASTQ files...")
    t1 = time.time()
    copy_fastqs(fastq_files, ref_file, datapath + 'fastq_files/')
    t2 = time.time()
    print("FASTQ files done. Time: %d".format(t2 - t1))
    print("Copying BAM files...")
    copy_bams(bam_files, datapath + 'bam_files/')
    t1 = time.time()
    print("BAM files done. Time: %d".format(t1 - t2))
    print("Copying VCF files...")
    copy_vcfs(vcf_files, datapath + 'vcf_files/')
    t2 = time.time()
    print("VCF files done. Time: %d".format(t2 - t1))
    
        
def data_test(fastq_files, bam_files, vcf_files, datapath):
    
    if not os.path.exists(datapath):
        os.mkdir(datapath)
    
    procs = []
    for vcf_file in cfg['vcf_files']:
        cmd = shlex.split('cp -r ' + vcf_file + ' ' + datapath + re.findall("/.*|$", vcf_file)[-1][1:])
        proc = sp.Popen(cmd, stdout=sp.PIPE, stderr=sp.PIPE)
        procs.append(proc)
    
    
    print("Copying Files...")
    copytime = 0
    while procs[-1].poll() == None:
        time.sleep(1)
        copytime += 1
        loadingstr = 'Copying'
        for i in range(copytime % 3):
            loadingstr += '.'
        print(loadingstr, end = '\r')
    print('                ', end = '\r')
    print('Done')
        
def process():
    
    cfg = json.load(open('config/process.json'))
    datapath = cfg['datapath']
    outpath = cfg['outpath']
    path_to_plink = cfg['path_to_plink']
    os.makedirs(outpath + 'plots/', exist_ok=True)
    
        
main()

In [ ]:
data(**(json.load(open('config/data.json'))))

In [ ]:
data_test(**(json.load(open('config/data-test.json'))))

In [ ]:
process(**(json.load(open('config/process.json'))))

# Part 1: Steps to work with 1000 Genomes files
- For fastq files, first must translate each individual into bam files then combine indidivudals into a vcf file using GATK
- For bam files, must combine individuals into vcf file using GATK
- For vcf file, can use plink2 to get PCA eigenvectors and eigenvalues then plot these PCA's and then eventually cluster 

# Part 2: Run.py
- Works on shorter chromosomes
- Takes a long time for longer chromosomes, need to nail down the right argument options
- Plots the PCA
- Wasn't sure what you meant with the data-test and process options? I used a config file to load data and process that.